<a href="https://colab.research.google.com/github/2020147544/Advances_in_Financial_Engineering/blob/main/chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labeling

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

p = print

In [3]:
def getDailyVol(close,span0=100):
    '''
    daily vol, reindexed to close
    - used to set default profit taking and stop-loss limits
    '''
    df0=close.index.searchsorted(close.index-pd.Timedelta(days=1))
    df0=df0[df0>0]
    df0=pd.Series(close.index[df0–1], index=close.index[close.shape[0]-df0.shape[0]:])
    df0=close.loc[df0.index]/close.loc[df0.values].values-1 # daily returns
    df0=df0.ewm(span=span0).std()
    return df0


SyntaxError: ignored

## Triple-Barrier Method

In [4]:
def applyPtSlOnT1(close,events,ptSl,molecule):
    
    '''
    Tripple-barrier labeling method
    ` Apply stop loss/profit taking, if it takes place before t1 (end of event)

    Input: 
    ` events: 
      — t1: the timestamp of vertical barrier
      — trgt: the unit width of the horizontal barriers, expressed in terms of absolute returns
    ` ptsl: a list of two non-negative float values
      - ptsl[0]: the factor multiplies trgt to set the width of the upper barrier
      - ptsl[1]: the factor that multiplies trgt to set the width of the lower barrier
    ` molecule: a list with the subset of event indices 

    Output: a Dataframe containing the timestamps at which each barrier was touched, [pt, s1, t1]
    ` 0 (inactive barrier) or 1 (active barrier)

    '''
    events_=events.loc[molecule]
    out=events_[['t1']].copy(deep=True)
    if ptSl[0]>0:pt=ptSl[0]*events_['trgt']
    else:pt=pd.Series(index=events.index) # NaNs
    if ptSl[1]>0:sl=-ptSl[1]*events_['trgt']
    else:sl=pd.Series(index=events.index) # NaNs
    for loc,t1 in events_['t1'].fillna(close.index[-1]).iteritems():
        df0=close[loc:t1] # path prices
        df0=(df0/close[loc]-1)*events_.at[loc,'side'] # path returns
        out.loc[loc,'sl']=df0[df0<sl[loc]].index.min() # earliest stop loss.
        out.loc[loc,'pt']=df0[df0>pt[loc]].index.min() # earliest profit taking.
    return out


In [5]:
def getEvents(close,tEvents,ptSl,trgt,minRet,numThreads,t1=False,side=None):
    '''
    Getting the time of first touch
     
    Input:
    ` tEvents: the pandas timeindex containing the timestamps that will seed every triple barrier 
      - the timestamps selected by the sampling procedures 
    ` minRet: the minimum target return required for running a triple barrier search
    ` numThreads: the number of threads concurrently used by the function  

    Output: 
    ` events: a Dataframe
      - events.index: event's starttime
      - events['t1']: event's endtime
      - events['trgt']: event's target
      - events['side'] (optional): the algo's position side
    '''
    #1) get target
    trgt=trgt.loc[tEvents]
    trgt=trgt[trgt>minRet] # minRet
    #2) get t1 (max holding period)
    if t1 is False:t1=pd.Series(pd.NaT,index=tEvents)
    #3) form events object, apply stop loss on t1
    if side is None:side_,ptSl_=pd.Series(1.,index=trgt.index),[ptSl[0],ptSl[0]]
    else:side_,ptSl_=side.loc[trgt.index],ptSl[:2]
    events=pd.concat({'t1':t1,'trgt':trgt,'side':side_}, axis=1).dropna(subset=['trgt'])
    df0=mpPandasObj(func=applyPtSlOnT1,pdObj=('molecule',events.index), numThreads=numThreads,close=inst['Close'],events=events,ptSl=ptSl_)
    events['t1']=df0.dropna(how='all').min(axis=1) # pd.min ignores nan
    if side is None:events=events.drop('side',axis=1)
    return events


## Labeling

In [12]:
def getBins(events,close):
    '''
    Labeling for side and size
    Input:
    ` events: a Dataframe
         
    Output:
    ` ret: the return realized at the time of the first touched barrier
    ` bin: the label as a function of the sign of the outcome 
      - Case 1: ('side' not in events): bin in (-1,1) <- label by price action (standard labeling)
      - Case 2: ('side' in events): bin in (0,1), i.e. pass or bet <- label by pnl (meta-labeling) 
    '''
    #1) prices aligned with events
    events_=events.dropna(subset=['t1'])
    px=events_.index.union(events_['t1'].values).drop_duplicates()  
    px=close.reindex(px,method='bfill')
    #2) create out object
    out=pd.DataFrame(index=events_.index)
    out['ret']=px.loc[events_['t1'].values].values/px.loc[events_.index]-1
    out['bin']=np.sign(out['ret'])  
    return out



In [14]:
def dropLabels(events,minPtc=.05):
    '''
    Apply weights, drop labels with insufficient examples

    '''
    while True:
        df0=events['bin'].value_counts(normalize=True)
        if df0.min()>minPct or df0.shape[0]<3:break
        print('dropped label',df0.argmin(),df0.min())
        events=events[events['bin']!=df0.argmin()]
    return events